# Use pytorch in distance


In [ ]:
import numpy as np
import torch

samples = []
from scipy.stats import multivariate_normal
for i in range(78400):
    x = np.random.rand(19)
    y = multivariate_normal.pdf(x, mean=2.5, cov=0.5);

samples.append(y)



In [14]:
from torch import nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(19, 128)
        self.fc2 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 3)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
#         x = F.softmax(x, dim=1)
        
        return x
    
model = Network()


In [22]:
from torch import optim

criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.003)
# criterion = nn.CrossEntropyLoss()


In [ ]:
epochs = 5
initial = np.random.rand(19)
sample_ = torch.from_numpy(initial).float()
output_ = model(sample_).data

for e in range(epochs):
    running_loss = 0
    for sample in samples:
        sample = torch.from_numpy(sample).float()
        optimizer.zero_grad()
        output = model(sample)
        loss = criterion(output, output_)
#         loss = torch.div(output, output_, out=None)
        loss.backward()
        optimizer.step()
        output_ = output.data
        
        running_loss += loss.item()
    else:
        print("Training loss: " + str(running_loss/len(samples)))